In [16]:
class MulLayer():
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y
        return round(out, 3)
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

class AddLayer():
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x+y
        return round(out, 3)
    
    def backward(self, dout):
        dx = dout*1
        dy = dout*1
        return dx, dy

# Mul

In [17]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
final_price = mul_tax_layer.forward(apple_price, tax)

print(final_price)

220.0


In [18]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


# Add

In [15]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
final_price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

print(final_price)
print(dapple, dapple_num, dorange, dorange_num)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0


# Activation Function

In [28]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x<=0) # 0보다 작은 idx: True 
        out = x.copy()
        out[self.mask] = 0 # 0이하는 0
        
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 /(1 + np.exp(-x))
        self.out = out
    
    def backward(self, dout):
        dx = dout * (1-self.out)*self.out
        return dx

In [25]:
x = np.random.randn(2,3)
mask = (x<=0)
mask

array([[ True,  True, False],
       [False, False, False]])

In [27]:
x[mask] = 0
x

array([[0.        , 0.        , 0.67697768],
       [0.56784062, 0.31223027, 0.87828589]])

# Affine

In [30]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

In [36]:
X = np.random.rand(4, 3)
W = np.random.rand(3, 2)
b = np.random.rand(2,)
A = Affine(W, b)
dout = A.forward(X)
A.backward(dout)

array([[2.62026361, 2.16191991, 1.52630749],
       [2.24404282, 1.91726943, 1.3027799 ],
       [2.07809633, 1.75255446, 1.20796653],
       [2.91846741, 2.41847404, 1.69931169]])

In [68]:
np.random.rand(2,10).

20

In [65]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

In [69]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(sefl, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

# Backpropagation
- 수치미분 vs 오차역전파
- 느림 - 빠름

# Neural net
- 가중치
- layer 생성

In [ ]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        
        #가중치 초기화
        self.params = {}
        
        self.params['W1'] = weight_init_std * \
        np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        
        self.params['W2'] = weight_init_std * \
        np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층생성
        # 레이어 각각에 W, b 존재
        # 각 레이어에 forward, backward method 존재
        # 외부 입력 데이터는 X
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params)
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
    
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x # probability matrix
    
    def loss(self, x ,t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t) # loss int
    
    def accuracy(self, x, t)
        y = self.predict(x) # mat
        y = np.argmax(y, axis=1)
        if t.ndim != 1: #one_hot
            t = np.argmax(t, axis=1)
        accuracy = np.sum(y==t)/float(x.shape[0])
        return accuracy
        
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t) # loss 계산용도 function
        
        grad = {}
        grad['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grad['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grad['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grad['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grad
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())